# INLP - Assignment 2
## Harshavardhan P - 2021111003

In [22]:
import torch
import torch.nn as nn
import conllu
import pandas as pd

## Importing Dataset

In [23]:
# import conllu from dataset paths
dataset_path_train = 'ud-treebanks-v2.13/UD_English-Atis/en_atis-ud-train.conllu'
dataset_path_dev = 'ud-treebanks-v2.13/UD_English-Atis/en_atis-ud-dev.conllu'
dataset_path_test = 'ud-treebanks-v2.13/UD_English-Atis/en_atis-ud-test.conllu'

dataset_train = conllu.parse_incr(open(dataset_path_train))
dataset_dev = conllu.parse_incr(open(dataset_path_dev))
dataset_test = conllu.parse_incr(open(dataset_path_test))

print("Number of sentences in training dataset: ", len(list(dataset_train)))
print("Number of sentences in dev dataset: ", len(list(dataset_dev)))
print("Number of sentences in test dataset: ", len(list(dataset_test)))

Number of sentences in training dataset:  4274
Number of sentences in dev dataset:  572
Number of sentences in test dataset:  586


In [26]:
# bring the pointer back to the beginning of the file
dataset_train = conllu.parse_incr(open(dataset_path_train))
dataset_dev = conllu.parse_incr(open(dataset_path_dev))
dataset_test = conllu.parse_incr(open(dataset_path_test))

# convert this data to a pandas dataframe
def conllu_to_pandas(dataset):
    data = []
    for tokenlist in dataset:
        for token in tokenlist:
            data.append(token)
    return pd.DataFrame(data)

df_train = conllu_to_pandas(dataset_train)
df_dev = conllu_to_pandas(dataset_dev)
df_test = conllu_to_pandas(dataset_test)

print(df_train.head())

   id  form lemma  upos  xpos  \
0   1  what  what  PRON  None   
1   2    is    be   AUX  None   
2   3   the   the   DET  None   
3   4  cost  cost  NOUN  None   
4   5    of    of   ADP  None   

                                               feats  head deprel  deps  misc  
0                            {'PronType': 'Int,Rel'}     0   root  None  None  
1  {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3...     1    cop  None  None  
2                                {'PronType': 'Art'}     4    det  None  None  
3                                 {'Number': 'Sing'}     1  nsubj  None  None  
4                                               None     7   case  None  None  


In [29]:
# create vocabulary of the words in the training set
vocab = df_train['form'].unique()
print("Number of unique words in the training set: ", len(vocab))

Number of unique words in the training set:  863
